
<span type="title">Servlet and JSP Intro</span> | <span type="update">2018-09-16</span> - Version <span type="version">1.9</span>
    
    
<span type="intro"><p class="card-text">本章主要对 Servlet 和 JSP 有一个概要性的介绍。在第一部分，介绍了HTTP的URL、GET/POST方法、Response响应码、HTML的主要格式、Web服务器和应用服务器的工作流程（包括web.xml文件配置、动态参数加载、属性监听和线程安全、Servlet生命周期、JSP派发等）。</p><p class="card-text">在第二部分，着重介绍了Session，包括Session本身的用法、URL重写、Cookies对象以及常见的HttpSessionListener、HttpSessionActivationListener、HttpSessionBindListener、HttpSessionAttributeListener及其使用方法。</p><p class="card-text">在第三部分，着重介绍了JSP以及基本语法和工作原理、流程，包括表达式、声明、scriplet、指令的使用，以及 URL和属性的配置、jspInit的重载、内置对象的使用。</p></span>

# HTTP Protocol

互联网通信基于 请求 和 响应 这两大动作。HTTP是一种协议，其在 RFC2616 草稿中规范，大概包括三大重要部分：客户端发送请求时的地址行和发送方法POST,GET等，服务端返回请求时的状态200，400，404等、返回请求时的报文类型以及报文内容。其中报文内容由HTML语言进行标记。

WEB指的就是HTTP和HTML为主体的层的活动。其中互联网请求多对应HTTP的GET和POST方法，响应则多对应于200,300,400等相应类型以及text/html,text/gzip等返回类型和HTML格式或其他格式的文档。

HTTP是一种应用层的协议，其建立在IP/TCP的基础上，IP负责规划和寻找客户端和服务端，TCP负责在线路上分批传送信息以及保证信息的准确性和完整性。HTTP得到的是由TCP生成的完整的报文，其只规定了在多台计算机间传送文档的格式（HTML）以及传送时的一些问题（各种传送状态、传送文档的地址、格式等）。

## HTTP URL

HTTP定义了URI，其中包含URL和URN。URL的格式为：`协议名称:// 子级地址 + [.] 上一级地址 + [.] 主地址 + [.] + 域名后缀:[端口地址]/ 文档路径/ 文档`比如：`http://www.mazhangjing.com/index/hello.txt` 其中协议是 http，域名是 mazhangjing.com（包含主地址和其后缀）。主机名是 www，文档路径为 index 文件夹，文件为此文件夹中的hello.txt文档。其中子级地址可以省略：`http://mazhangjing.com` 文档路径和文档名称都可以省略，并且设置默认值。端口地址默认为80，可以另行指定。此外，URL还包含可选的查询字符串，当使用GET方法时可以少量的添加请求的信息。（见下文）

HTTP的端口是一个16位数，其中1-1023被保留用作它用。一般而言，用于特定用途的服务会默认使用特定的端口，这是一种约定俗成的标准，比如SSH的22端口，HTTP的80和8080端口，HTTPS的443端口和FTP的21端口。

## HTTP GET/POST

对于一个地址，可以使用GET和POST方法，其中GET多用于超链接点击，其可以传送少量参数，并且会显示在地址栏中，容易暴露，比如：`http://www.mazhangjing.com/weather?location=wuhan&time=now`，当对于 `http://www.mazhangjing.com/weather` 发送GET方法时，如果传递了参数，会显示在URL上，这里明显看出参数有两个，location是wuhan，time是now，各个参数用&分隔，参数和文档用？分隔。

POST方法则不会暴露这些信息，并且可以传送大量的数据到服务器，比如文件上传等，因此多用于登录，提交信息等。POST传递参数和GET一样，除了不会直接显示之外，使用F12开发者工具（Chrome/FireFox）可以查看POST发送的数据，注意其没有被显示在地址栏中。

GET和POST的区别不仅仅在于传递的参数可见/不可见，或者POST能传递大量的数据，而GET只能传递少量并且裸露的数据。区别在于，GET方法是原子性的，不论GET多少次，只要参数不变，那么GET的内容不变。而POST则更倾向于是非原子性的，它被看作需要修改数据库上的一些信息，当多次POST时，浏览器一般会弹出警告，对于设计不良的WEB程序，多次POST会导致数据被重复更改。POST是HTTP方法中唯一非幂等的方法。


## HTTP RESPONSE

当我们发送GET/POST请求时，会收到来自服务器返回的响应，称之为 HTTP RESPONSE,其中重要的信息是响应代码和类型以及返回的文件格式以及文件内容本身。比如：`HTTP1.1 200 OK`，代码200，正常响应请求。`Content-Type：text/html`，表示返回的是一个HTML文档。


# HTML Standard

不同于XML，HTML是一种已经预先定义好的格式标记语言，其中 `<form>` 域定义了输入单位-表单。`<input>` 域定义了输入框、提交按钮、单选按钮、下拉菜单等选项，一般被包含在 form 域中。form 域可以有一些其余 html 元素用于提供辅助信息。下面展示了一个用于登陆的 html 文档：
    
```html
<html>
<head><title>Login</title></head>
<body>
	<h1>Login</h1>
	<p>login system, please type name and passwd, press submit button.</p>
	<form method="POST" action="/login">
		name: <input type="text" name="name"/><br/>
		passwd: <input type="text" name="passwd"/><br/><br/><br/>
		<input type="SUBMIT"/>
	</form>
</body>
</html>
```

上述的HTML登录代码中，当按下 submit 时，浏览器会收集 form 中的两个 input 域中的信息，作为字符串，使用 method 属性规定的方法，发送到 form 的 action 属性定义的路径上。上述实际post的参数为 `name=string1&passwd=string2`。method 方法可以规定为 POST 或者 GET，不过对于登录这种需求，POST更为常用。

# Web Server

## HTTP 请求的响应流程

当我们指定GET一个网址，客户端会使用HTTP协议发送这个请求，交给TCP到IP，然后经过物理层，找到URL中对应的主机，逆向从IP到TCP到服务端的HTTP，之后WEB服务器监听的端口监听到请求，找到URL路径上的文件，处理可能有的参数，并且返回一个响应，这个响应包含了响应的状态和状态码，以及返回的文档类型以及文档本身。返回的流程和获得的流程相反，从服务端的HTTP到IP到TCP，然后逆向到请求端的TCP到请求端的IP到请求端的HTTP，这样客户端就收到了服务器的文档。

注意，服务端是如何知道客户端的地址的呢？这是通过IP协议规定的，当发送请求时，服务端会得到请求主体的信息，包括其请求的时间以及请求的内容，还有请求主体的IP地址，知道这个IP，就可以返回到那台发送请求的主机上了。

WEB 服务器，比如 Negix 和 Apache Web Server，擅长并发处理静态请求，但是对于动态请求就无能为力。这意味着，所有的客户从WEB服务器上获取到的都是相同的文档，这显然是难以接受的。因此，我们需要动态返回自定义的文档这一功能，这是通过应用服务器完成的。

WEB服务器决定将动态请求交给应用服务器，并且通过SOCKET进行通信，这样的话，其只用处理静态请求，而动态请求被应用服务器处理后交给WEB服务器，作为整体文档返回用户。

## App Server 主要功能

应用服务器主要提供五大功能：

- 通信支持：Socket相关的网络处理、监听端口和创建流等

- 生命周期支持：Servlet的创建、调用和回收

- 多线程支持：一个用于管理Servlet连接的多线程池

- 声明方式安全：使用XML管理描述控制符并且简化开发流程，更加安全

- JSP支持：JSP文档解析和Java代码翻译

## Servlet 和 JSP 技术

常见的应用服务器有PERL/Python常用的CGI统一网关接口，也有JAVA标准的Servlet标准。Servlet相比较CGI具有很多优点，比如：

Servlet不会对于每次请求启动一个进程，当第一次启动后，它就保持开启状态，并且多个指向同一Servlet的请求可以由同一个Servlet的单独线程来处理。

如果这些请求是并发的，那么Servlet容器的线程池会保证资源消耗，并且进行并发控制。

## MVC 设计模式

基本的Java Servlet & JSP 需要使用 MVC 设计模式，其中请求从容器的XML文件中找到对应的Servlet，之后Servlet进行Controller的角色处理，后通知模型改变状态，将请求转发给JSP，JSP改变视图并且进行相应。下面是一个典型的MVC，其中当用户产生请求，此请求被容器派发到匹配的Servlet，这个Servlet充当Controller的角色，通过解析POST的参数，然后交给Model进行处理，并且将其结果附着在request的属性上，交给JSP去解析和展示。

index.html

```html
<body>
    <h1>Beer Select Page</h1>
    <form action="SelectBeer.do" method="post">
        <p>Select your favorate beer color</p>
        Color: <select name="color" size="1">
        <option value="light">light</option>
        <option value="amber">amber</option>
        <option value="brown">brown</option>
        <option value="dark">dark</option>
        </select>
        <input type="submit"/>
    </form>
</body>
```

web.xml
```xml
<servlet>
    <servlet-name>select_beer</servlet-name>
    <servlet-class>BeerSelect</servlet-class>
</servlet>
    
<servlet-mapping>
    <servlet-name>select_beer</servlet-name>
    <url-pattern>/SelectBeer.do</url-pattern>
</servlet-mapping>
```

BeerSelect.class

```java
public class BeerSelect extends HttpServlet {
    @Override
    protected void doPost(HttpServletRequest req, HttpServletResponse resp) throws ServletException, IOException {
        req.setAttribute("color",
                new BeerModel().getBeer(req.getParameter("color")));
        RequestDispatcher dispatcher = req.getRequestDispatcher("result.jsp");
        dispatcher.forward(req,resp);
    }
}
```

BeerModel.class

```java
public class BeerModel {
    List<String> getBeer(String color) {
        switch (color) {
            case "light":
            case "amber":
                return new ArrayList<>(Arrays.asList("Qingdao","ChunSheng"));
            case "brown":
            case "black":
            default:
                return new ArrayList<>(Arrays.asList("Black-bear","HuangShan"));
        }
    }
}
```

result.jsp

```jsp
<%
    List choose = (List) request.getAttribute("color");
    Iterator iterator = choose.iterator();
    while (iterator.hasNext()) {
        out.print("<br>Try:" +iterator.next() + "!");
    }
%>
```

# Container: XML Config

一般而言，常见的Servlet和JSP容器是Tomcat。TOMCAT主要负责Servlet的生命周期管理，URL映射等，使用Servlet必须装配在一个容器中才可以使用。TOMCAT并非是一个完整的Java EE容器，而是一个完整的Servlet & JSP 容器，tomcat的文件层次如下：

- WEB-INF 目录主要存放 web.xml 映射描述文件，classes 源代码文件，lib 共享的库文件。

其中 servlet 存放于 classes 文件夹下，对于 `com.mycompany.mypackage.MyServlet`，应该存放在 `/WEB-INF/classes/com/mycompany/mypackage/MyServlet.class.` 目录下。

对于 web.xml 文件，举例如下：

```xml
<web-app>
    <!-- 此应用的具体信息，比如标题和描述。每个应用指的是一个一级的 context-path，在一个主机中可以有无数个应用
    ，比如www.mazhangjing.com/app1 这是一个应用， .../app2 这是另一个应用。-->

    <display-name>My Web Application</display-name>
    <description> This is my first web application.
    </description>


    <!-- 在这里可以定义一些Servlet和JSP共用的上下文参数，使用下列语句获取此处的信息
         String value = getServletContext().getInitParameter("name");
    -->

    <context-param>
      <param-name>webmaster</param-name>
      <param-value>myaddress@mycompany.com</param-value>
      <description>
        The EMAIL address of the administrator to whom questions
        and comments about this application should be addressed.
      </description>
    </context-param>


    <!-- servlet 可以通过下列方式被调用，但是，更常用的方式是，通过URL地址的转发
         http://localhost:8080/{context-path}/servlet/{classname}
         下面的servlet定义了其名称、描述、响应的类地址、初始化的参数信息（值对）以及启动顺序
    -->

    <servlet>
      <servlet-name>controller</servlet-name>
      <description>something here</description>
      <servlet-class>com.mycompany.mypackage.ControllerServlet</servlet-class>
      <init-param>
        <param-name>listOrders</param-name>
        <param-value>com.mycompany.myactions.ListOrdersAction</param-value>
      </init-param>
      <load-on-startup>5</load-on-startup>
    </servlet>

    <servlet>
      <servlet-name>graph</servlet-name>
      <description> something </description>
    </servlet>


    <!-- 每个 servlet 最好提供一个 servlet-mapping 的元素，用于指定servlet和地址的映射关系。
         其中必要包含servlet-name和url-pattern两者。url-pattern中定义的请求，一旦被匹配，则
         自动转向servlet进行处理。注意，可以使用通配符匹配后缀，可以指定精确的路径和文件，可以
         只指定路径。（这里有很多坑需要注意，这里没有正则表达式灵活）
    -->

    <servlet-mapping>
      <servlet-name>controller</servlet-name>
      <url-pattern>*.do</url-pattern>
    </servlet-mapping>

    <servlet-mapping>
      <servlet-name>graph</servlet-name>
      <url-pattern>/graph</url-pattern>
    </servlet-mapping>
    
</web-app>
```

## initParameter 参数动态加载

对于不想要硬编码在 Servlet 或者 JSP 中的数据，也不想放到数据库中的数据，可以放在XML描述文件中。放在这里的数据可以通过 getContext 或者 getConfig 的 initParameter 参数获得。但是，TOMCAT需要热重启来加载这些数据（因为这些数据在初始化servlet后被加载，之后再也不被访问）。关于使用 config 还是 context，如果数据是单个 servlet 使用，则使用 config，否则使用 context。

换句话说，这些参数是“常量”。

```java
out.print("USER: " + getServletConfig().getInitParameter("username"));
out.print("APP: " + getServletConfig().getServletContext().getInitParameter("appname"));

//USER: null APP: muninn
```

尤其需要注意，getServletConfig() 中必须使用对应的 servlet 的参数。上述代码为 null 的原因是，我们的 servlet 将 resp 派发给了 jsp，这个 jsp 是另一个 servlet，因此，自然获取不到数据。此外， getServletContext是由 getServletConfig 方法的 Config 对象提供的。

# Controller: Servlet 

Servlet 常用的类是 HTTPServlet，需要重载这个类，提供doGET, doPost方法来截获请求并且进行处理。举例如下：

```java
import java.io.*;
import javax.servlet.*;
import javax.servlet.http.*;

public class HelloWorld extends HttpServlet {

    public void doGet(HttpServletRequest request, HttpServletResponse response)
    throws IOException, ServletException
    {
        response.setContentType("text/html");
        PrintWriter out = response.getWriter();
        out.println("<html>");
        out.println("<head>");
        out.println("<title>Hello World!</title>");
        out.println("</head>");
        out.println("<body>");
        out.println("<h1>Hello World!</h1>");
        out.println("</body>");
        out.println("</html>");
    }
}
```

这个Servlet程序对response对象设置了类型，然后对于输出流添加了HTML文档，当容器检测到对于这个Servlet调用时，会进入 doGet 进行处理，返回 response 的文档格式和文档内容。URL到Servlet的映射可以通过注解和XML标记文件指定。


## Servlet 对象的生命周期

在处理流程中，我们说到，servlet对象对于每个请求派发一个线程，而不是启动一个实例。对于一台电脑的多个请求会对应多个线程，对于分布式，则每个JVM包含一个实例，然后再分成多个线程。

servlet对象的创建如下：

- 首先找到.class类，容器使用classLoader进行加载
- 实例化servlet对象，容器负责，你需要指定启动顺序
- servlet的init方法，像是数据库连接在此处进行
- servlet的serve方法，当得到请求，会调用此处以继续调用goPost/goGet进行委托处理。
- servlet的destory方法，销毁前的准备，比如关闭文件流或者数据库

注意，init和destory只进行一次，对于多个请求，容器会多次在不同线程中调用serve方法进行处理。不要把任何代码放到servlet的构造器中去。因为这个时候很可能我们想要的数据还没有被初始化，比如上下文信息。

## Servlet 和 JSP 的处理流程

对于一个请求，TOMCAT容器的处理步骤为：容器首先创建一个 HttpServletRequest 和 HttpServletResponse 对象，然后调用 Servlet 的一个线程，调用 service() 方法，之后 service 方法根据POST还是GET调用 doPost() 和 doGet() 中的方法，并且传递 request 和 response 对象作为参数，方便我们进行处理，处理后，将其中的 response 方法的流返回，进行相应。此时，HttpServletRequest 和 Response 对象死掉，Servlet 的相关线程死掉。

需要注意，在MVC设计模式中，HTML的GET or POST 请求，在 Servlet 中，我们会从 req 中提取 parameter 参数，然后交给 Model 进行处理，得到的结果再交给 req，set 到 attritube 上。注意，parameter 指的是从HTML到Servlet的参数传递，默认为 String，但是Attritube则不同，其一般为方便JSP进行后期处理的Java类型，attritube 多用于 Model 交付结果给 JSP 进行页面展示的。

## servletConfig 和 servletContext

每个 servlet 对象都有一个 servletConfig 对象，此对象用于向servlet传递部署时的信息，这些信息在xml文件中，可以方便的进行更改。此外，servletConfig对象还可以用于访问 servletContext 对象，后者其实应该叫做AppContext，因为每个Web APP只有一个servletContext对象，此对象用于访问服务器信息，API版本信息，Web应用的参数以及路径下的其他可访问文件等。换句话说，servletContext像是一个App的外在环境背景。此对象必须通过servletConfig对象获得，servletConfig对象针对单一的servlet进行配置，传递每个servlet独特的参数（而不是APP全局的参数），以及提供对于servletContext的访问。

## HttpRequest 对象

HttpRequest 对象由容器生成，并且在调用 service -> doXXX() 方法时作为参数传入。当我们需要使用RequestDispicter时，一般会先 getParameter 交给 Model 处理后获得结果以Java类型 setAttritube 到 Request 对象上，由JSP负责解析。

Request 常见方法一般为端口相关、内容相关、附带的参数和属性的获取

- getLocalPort/RemotePort/ServerPort 获取本地相应端口/客户端端口/监听端口

- getContentLength 获取内容长度

- getInputStream 获取输入流（POST方法用来获取大规模二进制数据）

- getParameter/getParameterValues/getParameterNames 获取参数（指定名称，单一结果）/获取参数（指定名称，多个结果）/获取参数枚举

- getAttribute 用于JSP或者派发的属性获取，区别于 Parameter，类型为 Object？

以上是 ServletRequest 的方法，以下是 HttpServletRequest 扩展的方法，主要包含了url、query、method、header的HTTP头部属性查询支持以及cookies和session的支持。

- getContextPath 获取相对地址

- getQueryString 获取查询的参数信息

- getCookies

- getSession

- getHeader/getIntHeader 后者不过是str->int的语法糖罢了

- getMethod POST/GET

## HttpResponse 对象

Response 的方法相比 Request 的就简单许多，对于 ServletResponse 而言，常见的是 get/setContentType, getOutputStream/Writer 输出内容相关的方法，对于 HttpResponse 而言，多了 addCookie，addHeader，encodeURL，sendError，setStatus，sendRedirect这些添加头部、改变URL、添加Cookie、更改状态以及重定向的方法。


## MIME 数据读写 

Servlet 的 Response 是一个动态的发送，对于浏览器而言，我们不知道到底接收到的是什么，反正是数据流，因此需要使用MIME类型来标记，比如：

```java
response.setContentType("application/jar");
InputStream = getServletContext().getResourceAsStream("/something.jar");
OutputStream os = response.getOutputStream();
int read; byte[] bytes = new byte[1024];
while ((read = is.read(bytes) != -1)) {
    os.write(bytes,0,read);
} os.flush(); os.close();
```

注意上面使用了 servletContext 对象的 getResourceAsStream 方法获取文件，注意其MIME类型。此外，这个文件的路径，不能是相对路径。如果是绝对路径，则根目录指的是APP的根目录，而不是WEB-INF的根目录。 


## RequestDispatcher 请求派发 

对于 request，在 servlet 中进行处理后，可以交给 dispatcher 进行转交。这一过程使用：

```java
req.getRequestDispatcher("result.jsp").forward(req,resp); //相对路径对于当前Servlet相应时的URL-Pattern而言
```
来处理，这里需要注意，dispatcher 可以来自于 req 对象，也可以来自于 context : 

```java
getServletContext().getRequestDispatcher("/result.jsp").forward(req,resp); //绝对路径相对Web-APP对应的URL地址而言
```

**请求派发的相对和绝对URL查找**

不过，这样有点繁琐，因为反正 forward 要用到 req 对象。其次，它和 req 的 dispatcher 有区别，派发器的地址必须是/绝对路径开头，因为context对象没有相对路径的记录。而req则必然依附在一个servlet上，因此可以使用相对路径。一般使用 req 对象的属性得到派发器。

派发接受jsp、html和Servlet的URL映射作为参数。对于Servlet，其映射在DD中被规定（Url-pattern和类的对应关系）。而对于jsp和html，其映射默认被看作物理路径。需要注意，这不意味着Servlet使用物理路径也可以被找到，因为派发始终接受的是映射，不过恰好jsp和html的映射就是物理地址，而Servlet并不是。

相对路径指的就是当前Servlet的URL地址，绝对路径就是WEBAPP的URL地址（区分重定向绝对路径的Host地址）。如果需要转发到Servlet/JSP处理，而这个Servlet/JSP的URL正好属于当前Servlet的下一级URL，那么使用相对路径。否则对于Servlet，使用其URL-Pattern的地址，对于JSP和HTML，使用映射对应的物理地址。（如果被派发的Servlet URL为 /abc/foo，而当前Servlet URL 为 /abc/*，那么使用相对路径 foo 即可）

其次，可以使用 forward 和 include 进行派发。区别是，前者将权力完全移交，后者只是请求帮助，在处理后还会返回此Servlet。因此，对于 include，可以在当前 servlet 调用 getWriter，反正还是要返回，但是对于 forward，在当前 servlet 一旦调用 getWriter().flush 则会直接出错（因为已经派发，但是自己又要返回），一般常用的是 forward。

**派发的request对象不同于原本的request对象**

当进行派发时，request对象变化了，被派发的request对象并非派发前的request对象。通过 getAttritube("javax.servlet.forward/include.xxx") 来获取这些派发前的属性/字符串值。


## Redirect 重定向

对于 setRedirect 方法，需要注意，其必须在 getWriter/getOutputStream 之前被调用，否则会返回状态错误。此外，redirect接受string字符串，可选绝对路径或者相对路径。相对路径是相对于用户输入URL去除文件所指向的目录，绝对路径指的是相对于根域名的目录（而不是APP的目录）。

比如：访问 `www.mazhangjing.com/appname/folder_a/index.html` 此时指向相对目录 `folder_b/index.html` 则意思是 `www.mazhangjing.com/appname/folder_a/folder_b/index.html`。而如果现在是绝对目录，则意识是 `/folder_b/index.html` => `www.mazhangjing.com/folder_b/index.html` 注意，appname 这一层消失了。可以看到，对于重定向是基于URL的，要么是基于CWD，要么是基于根域名，和APP根目录没有关系。这和 context.getResourceAsStream 非常不同，后者的绝对目录是以 App 为根而不是以服务器地址为根。

# Listener: 属性和事件监听

## ServletContextListener 示例

我们说过，对于 servlet 而言，将数据库的初始化和销毁之类的工作放在 init 和 destory 方法中，在 servlet 构造时只会执行一次。但是，对于想要共享给多个 servlet 和 jsp 的数据库，我们就束手无策了。现在知道 context-param 是放置 string 字符串参数的地方，在 xml 中声明 context-param 可以让多个 servlet 共享字符串参数，但是对于 Java 对象，我们需要使用 “context-attritube”。 

没有这样一个地方（main方法）让我们初始化上下文属性的，因此，我们需要使用 listener，当监听到TOMCAT创建上下文，初始化APP时，开始我们的小动作。在监听器中，通过设置context-attritube，我们可以使用 context.getAttritube() 方法来获取这些 POJO 属性。并且多个 servlet 共用。listener 需要在 xml 中配置类的地址，以便 TOMCAT 初始化监听器并自动加入监视列表。

web.xml

```xml
<context-param>
    <param-name>breed</param-name>
    <param-value>something here</param-value>
</context-param>
<listener>
    <listener-class>com.mazj.dog.DogListener</listener-class>
</listener>
```

com.mazj.dog.Dog.class || 
com.mazj.dog.DogListener.class

```java
package com.mazj.dog;
public class Dog {
    String breed;
    Dog(String breed) { this.breed = breed; }
    public String getBreed() { return breed; }
    public void setBreed(String breed) { this.breed = breed;}
}
public class DogListener implements ServletContextListener {
    public void contextInitialized(ServletContextEvent sce) {
        String breed  = sce.getServletContext().getInitParameter("breed");
        sce.getServletContext().setAttribute("dog",new Dog(breed));
    }
    public void contextDestroyed(ServletContextEvent sce) { }
}
```

index.jsp

```jsp
<%@ page import="com.mazj.dog.Dog" %>
<p><%
    String res = ((Dog)getServletConfig().getServletContext().getAttribute("dog")).getBreed();
    out.print(res+"\n");
%></p>
```

## 常见的8大监听者

针对 request 的监听者有：

- ServletRequestListener - ServletRequestEvent - 针对请求时间的监视（用于日志记录等）

- ServletRequestAttributeListener - ServletRequestAttributeEvent - 针对请求属性的增删改的监视

针对 session 的监听者分为两类，在session确认前的统计和跨JVM激活事件以及在session确定后的触发和属性更改事件。

- HttpSessionListener - HttpSessionEvent - 针对活动会话建立的统计管理（并发）

- HttpSessionBindingListener - HttpSessionBindingEvent - 针对会话被绑定或者删除时的通知（用于属性类的失效）

- HttpSessionActivationListener - HttpSessionEvent - 针对会话在JVM上迁移的通知

- HttpSessionAttributeListener - HttpSessionBindingEvent - 针对会话属性增删改的监视

针对 context 的监听者有：

- ServletContextListener - ServletContextEvent - 针对上下文建立和销毁时的通知

- ServletContextAttributeListener - ServletContextAttribute - 针对上下文属性增删改时的通知

注意，监听器监听属性的修改，Event返回的是旧值。

**顺序已知的监听** 不同于监听者模式，因为DD中监听的顺序一定，所以监听者被通知的顺序也得到保证，这一点和JAVASE某些实现不同。


## 线程安全的属性处理

对于 context 的属性，其可以进行添加和删除，作用域存在于 APP 初始化到 APP 终结为止，所有APP内的 servlet 和 jsp 甚至 listener 均可访问。 对于 Session 的属性，其也可以进行添加和删除，作用域存在于用户的会话到期为止，所有在到期前被调用的 servlet 和 jsp 均可以访问。对于 request，只有特定的 servlet 和其派发 request 可以访问，作用域一直持续到返回视图为止。

除了局部变量和请求属性，所有的用户会话属性以及上下文属性都不是线程安全的，因为每次请求会使用servlet的不同线程而不是不同实例，如果在set之后，get之前被CPU切换到另一个servlet线程或者另一个servlet实例，那么会发生同步错误。因此，经常对于set进行同步化来避免线程问题：

```java
synchronized (getServletContext()) {
    getContext().setAttribute("boo","res1");
    out.println(getContext().getAttribute("boo"));
}
synchronized (req.getSession()) {
    session.setAttribute("foo","42");
    out.println(session.getAttribute("foo"));
}
```

注意，synchronized 需要对于 Object 加锁，也就是 session 和 getServletContext 加锁。其次，不对 doGet 方法加锁的原因是，不只因为同一 servlet 的不同线程可能访问属性，不同servlet实例也有可能同时对于属性进行读写。因此，对于属性的宿主加锁是唯一可以保证线程安全的方案。

# Session

当客户端向服务器发送请求，服务器无法认识客户端，因此需要使用凭证信息来标明是何者发送的请求，使用的技术叫做 HTTP Session。Session 可以基于 URL，或者是Cookie，或者是SSL。常用的方式是Cookie。基于URL安全性很差，容易暴露以及被劫持。

## Cookie

cookie 是一种在 HTTP 协议中用来保存 key-value 字段值的一个参数，其存在于 Header 中。 cookie 只能保存有限的字符串信息，并且在每次 POST/GET 时都会添加到 header 中进行发送，用来标记客户端行为和状态信息。

基于 cookie 的 session是一种特殊的 cookie，这个时候，我们将更多的客户端行为和状态保存在服务器上，给这个状态一个 session id 凭证信息。然后客户端的 cookie 中保存这样一个 session id信息。这样的话，每次 post/get 都可以从 cookie 中抽取 session id，然后找到服务器对应 id 的复杂客户端状态了。

你可以看到设置过 session 后，cookie 字段会多出一个：Cookie: JSESSIONID => 8E66432316B23ACD89A2559C8D74E669 这样的字段，这是用来标记服务器上的Session对象的字段。其中客户端在每次请求时都会携带这样的字段，服务器借助这个字段来获取会话的属性信息以及识别用户。**需要注意的是，Servlet规范规定Cookie必须使用JSESSIONID来设置Session，而不能使用其余字段。**

我们可以直接使用 cookie，而不用容器自动管理的 session 对象。session 一般是会话相关，保存用户口令和之间的操作等。而cookie则可以是用户相关，这意味着cookie可以设置单独的过期时间，保存更加长久的用户信息，比如用户名之类。大部分的 session 是基于 cookie 实现的，cookie 能提供比 session 更长久的简单信息。

cookie的使用方法如下：

```java
Cookie color = new Cookie("color",req.getParameter("color"));
color.setMaxAge(10*6);
boolean find = false;
for (Cookie cookie : req.getCookies()) {
    if (cookie.getName().equals(color.getName())) {
        cookie.setValue(color.getValue()); find = true; break;
    }
} if (!find) resp.addCookie(color);
```

Cookie可以由 req 对象的 getCookies 获得，是一个数组，需要注意，必须遍历这个数组来匹配。此外，没有 setCookies 方法，如果需要添加一个新的 Cookies，那么对于 resp 对象，使用 addCookies 来添加一个 Cookie。 Cookies 可以使用 getName、getValue 访问键值对。使用 setValue 来设置其新值。此外，其可以单独设置过期时间。


## 基于 Cookie 的 Session

Serlvet API 简化了基于 cookie 的 session的使用，我们只用操纵 Session 对象即可，其余工作：设置 cookie 交给容器去做。基本的Session使用方法如下，对于一个 request 的请求，创建 Session/获取 Session 对象。

```java
HttpSession session = req.getSession();
//req.getSession(false); 这种方法不会自动新建session，只会得到null或者存在的session
if (session.isNew()) { System.out.print("Session is new"); }

session.setMaxInactiveInterval(10 * 60);
session.setAttribute("name","User");
session.setAttribute("dog",new Dog("wooo"));

System.out.println(
        String.format(
                "\n++++++++ Session info +++++\n" +
                        "Session id: %s\n" +
                        "Create time: %s\n" +
                        "Last access time: %s\n" +
                        "Interval time: %s s\n" + "+++++++++++++++++++++++\n"
                , session.getId(), session.getCreationTime(),
                session.getLastAccessedTime(), session.getMaxInactiveInterval())
        //也可以在DD中设置 session-config 不过，那里的时间单位是min。
);

//session.invalidate();
```

注意到，使用 getSession 如果不存在 Session，则自动会创建，为了避免这种行为，使用 getSession(false)。使用 session.isNew() 判断是否刚创建的 Session 对象。session 可以被设置过期时间，单位为s，可以添加key-value的属性，可以为POJO类型格式，反正是在服务端存储。客户端只会携带 session 的 id 字段，用来作为识别凭证。

Session 可以获取下列属性：id、creationtime、lastaccessedtime、maxinativeinterval。其中创建和最后访问事件单位为ms。

Session 的过期时间，以及 url、cookies、tcp的选择，可以在 DD 中定义，注意，这里定义的是min分钟！！！ 任何情况下，设置-1表示永不过期。

```xml
<session-config>
    <session-timeout>10</session-timeout>
</session-config>
```

## 基于 URL 的 Session

Session 除了使用 cookies，还可以使用 URL，一般情况下，不推荐这样使用，但是对于拒绝 Cookie 的用户，则必须这样来进行识别。

req.encodeURL() 和 req.encodeRedirectURL() 方法可以用来添加JSESSION凭证字符串，区别是，前者会自动将相对URL变成绝对URL，并且添加凭证，而后者用于重定向，只是单纯的添加凭证。这两个方法返回String字符串，需要动态编码到HTML/JSP中。

在JSP视图中，获得Cookies、URL、Session 的信息：

```java
out.print(String.format("\nSession id: %s\nAttribute name: %s\nAttribute dog: %s\n",
        session.getId(),session.getAttribute("name"),
        ((Dog)session.getAttribute("dog")).getBreed()));

session.invalidate();
out.print(response.encodeURL("/go"));

for (Cookie cookie : request.getCookies()) {
    out.println(String.format(
            "\nCookie: %s => %s\n",cookie.getName(),cookie.getValue()));
}
```

```
Session id: B4AC8DD468586C97326D18351E0F777A 
Attribute name: User 
Attribute dog: bound!!!! 
/go 
Cookie: color => light
Cookie: JSESSIONID => 8E66432316B23ACD89A2559C8D74E669
```

## SessionListener

Session 可以添加很多的监听器，比如新建或者销毁 Session 时，可以使用 SessionListener 和 SessionEvent 对象，下面展示了如何使用此事件来计数当前在线人数。

```java
public class Counter implements HttpSessionListener {
    public static int count;
    @Override
    public void sessionCreated(HttpSessionEvent se) {
        System.out.println("\nnew Session Add! Listener report: " + se.getSession().getId());
        count++;
    }
    public static int getCount() { return count; }
    @Override
    public void sessionDestroyed(HttpSessionEvent se) {
        count--;
    }
}
```

## SessionAttributeListener

这个监听器可以监听属性改变时的变化，使用 SessionBindingEvent 来监测属性被绑定、删除和替换的事件。

```java
public class AttributeListener implements HttpSessionAttributeListener {
    
    public void attributeAdded(HttpSessionBindingEvent se) {
        System.out.println("\n" + se.getName() + " has be added");
    }

    public void attributeReplaced(HttpSessionBindingEvent se) {
        System.out.println("\n" + se.getName() + " has be replaced");
    }

    public void attributeRemoved(HttpSessionBindingEvent se) {
        System.out.println("\n" + se.getName() + " has be removed");
    }
}
```

## SessionBindingListener

## SessionActivationListener

这个监听器可以监听一个属性对象被绑定和取消绑定时的消息，使用 SessionBindingEvent 来监测。

```java
public class Dog implements HttpSessionBindingListener, HttpSessionActivationListener {
    String breed;
    public Dog(String breed) {
        this.breed = breed;
    }
    public String getBreed() {
        return breed;
    }
    public void setBreed(String breed) {
        this.breed = breed;
    }

    public void valueUnbound(HttpSessionBindingEvent event) {
        System.out.println("Value unbound! by session : " + event.getSession());
        setBreed("unbound!!!!");
    }

    public void valueBound(HttpSessionBindingEvent event) {
        System.out.println("Value bound! by session : " + event.getSession());
        setBreed("bound!!!!");
    }
}
```

需要注意，这个监听器不需要向DD注册，当取消绑定时，会自动通知。

对于 HttpSessionActivationListener 适用于多个 JVM 会话迁移的情况，当服务器1创建了一个会话，但是这个会话现在由服务器2来服务，那么服务器2会从1上序列化传送会话的各个属性，当发生这种情况时，监听器可以进行冻结前和恢复后的操作。一般较少使用。

当且仅当属性能够序列化或者被ActivationListener通知后进行序列化，否则属性会丢失。

以下是上述Listener的控制台输出信息：

```
new Session Add! Listener report: B4AC8DD468586C97326D18351E0F777A
Session is new
name has be added
Value bound! by session : org.apache.catalina.session.StandardSessionFacade@456c64f
dog has be added

++++++++ Session info +++++
Session id: B4AC8DD468586C97326D18351E0F777A
Create time: 1537417936005
Last access time: 1537417936005
Interval time: 600 s
+++++++++++++++++++++++

name has be removed
Value unbound! by session : org.apache.catalina.session.StandardSessionFacade@456c64f
dog has be removed
```

# Viewer: JSP 

对于 JSP 而言，它类似于一个带壳的 Servlet，容器会有一个 JSP 的解析器用于对JSP文档进行解析，然后使用一个Servlet实例响应对于JSP的请求。JSP因为将Java代码嵌入到HTML中，因此使用更加方便。在早期，流行直接通过客户端URL对于JSP文件位置进行访问，JSP会生成Servlet实例并且进行响应，这避免了在Servlet中写HTML代码的转义问题。

但是，这样有安全风险，因此现代程序往往让JSP作为view层，客户端URL通过servlet处理并且转发到JSP文件（以及另一个servlet）进行其余工作后返回视图。JSP现在对于客户不可见。不过，要知道，不用在web.xml中配置映射，jsp文件可以直接被URL定位符解析并且定位到文件位置，自动进行初始化和响应。

## HttpJspBase 对象

JSP 文件本质上是一种“默认URL映射和物理地址映射对应” 的 Servlet。当我们请求一个 JSP 文件的时候，JSP 文件首先被在 DD 中定位和找到，然后使用容器的 JSP 加载器进行转换，转换成为 Servlet.java -> Servlet.class。之后像正常 Servlet 一样，这个类被初始化构造器、调用init方法、被多个线程调用service方法提供服务，最后调用destroy方法终止服务。JSP 文件只经过一次转换，之后不再使用，一直调用转换后的 Servlet 对象。对于 TOMCAT 而言，此 Servlet 的位置一般在： `C:\Tomcat(TOMCATFolder)\work\Catalina\localhost(HostFolder)\ROOT(APPFolder)\org\apache\jsp` 下。

这个 Servlet 被命名为 原始jsp文件名_jsp.class。比如说：

```java
public final class index_jsp extends org.apache.jasper.runtime.HttpJspBase
    implements org.apache.jasper.runtime.JspSourceDependent, org.apache.jasper.runtime.JspSourceImports
```

JSP 文件一般提供以下能力：导入package的page指令、执行Java代码的scriptlet、用于变量和方法初始化的声明、用于输出String的表达式。对于包导入而言，会在这个Servlet最上面进行导入。对于变量和方法的初始化而言，会在类的层级下进行设置，对于代码执行而言，会在service方法下执行，对于表达式输出而言，会调用输出流write/println对应的HTML元素。

对于 org.apache.jasper.runtime.HttpJspBase 类而言，其提供了三个重要的方法：

- jspInit() 在JSP初始化后提供一个一次性的处理工作的地方，比如数据库连接之类

- jspDestory() 在JSP被销毁的时候，提供一个处理的窗口，比如关闭数据库之类，和init一样，只被调用一次。

- _jspService() 任何情况下都不应该覆盖，容器会自动执行service方法并且提供服务。其参数包括req和resp，每次客户请求都会使用这个方法。

这里没有区分POST和GET，因为service会自动在方法内进行判断和处理。

## JSP语法简要介绍

**Scriptlet** 两个百分号和尖括号，表示这是一个scriptlet，里面放置纯Java代码，在jspService中顺序执行。
```
<%
    int a = 20;
    System.out.println("a is" + a);
%>
```

**指令** 第一个百分号后紧接着@，表示这是一个指令，后面的page表示这是一个page指令，此外还有include、taglib指令。
信息在指令的属性中设置，字符串形式表示，page指令主要处理比如是否允许会话、文件类型、字符串编码、导入类之类
```
<%@ page import="com.mazj.dog.Dog,com.mazj.cat.*" 
         session="true" contentType="text/html;charset=UTF-8" %>
```

**表达式** 第一个百分号后紧接着=，表示这是一个表达式，里面的内容会被 strip 后放在 out.println() 中，在jspService方法中执行，任意可以被打印的东西都可以放在其中执行（任意有toString方法的对象都可以）
```
<%= 20*88 %>
```

**声明** 第一个百分号后紧接着！表示这是一个声明，声明放在HttpJspBase的类下执行，这里可以用来定义变量和方法，因为一旦被放置在jspService中，每次调用则会被覆盖。
```
<%! Dog dog = new Dog();
    void setDog() {
        //dosomething here
        return;
    }
%>
```

**注释** 以下是三种可用的注释形式：
```
<!-- --> HTML注释，最后会被显示在HTML中，作为注释存在。此外，可以动态生成HTML注释： <!-- <%= 20*30 %>--> = <!--600-->
<%-- --%> JSP注释，可以用在任意位置，不会显示在HTML中。
// Java注释，只能用于声明和scriptlet中。
```

## URL配置和参数获取

因为JSP默认物理地址和URL映射对应，因此可以直接访问JSP文件物理路径即可获取服务。但是最好还是要写成这样：

```xml
<servlet>
    <servlet-name>Hello</servlet-name>
    <jsp-file>/hello.jsp</jsp-file>
    <init-param>
        <param-name>Database</param-name>
        <param-value>waterFire</param-value>
    </init-param>
</servlet>

<servlet-mapping>
    <servlet-name>Hello</servlet-name>
    <url-pattern>/hello</url-pattern>
</servlet-mapping>
```

注意到这里和之前Servlet不同之处在于Servlet提供servlet-class字段，而jsp提供jsp-file字段。当然，如果不这么写，直接访问/hello.jsp也可以，不过我们还是要避免直接对于jsp文件URL映射的访问，这样安全性较差。此外，使用DD的好处在于，可以访问初始的参数等其余信息，而单纯的jsp页面则不可以。

## JSP 属性组的 DD 配置

此外，我们可以额外的配置 jsp 属性组：

```xml
<jsp-config>
    <jsp-property-group>
        <description>All jsp page use this property group</description>
        <display-name>all_jsp</display-name>
        <url-pattern>*.jsp</url-pattern>
        <el-ignored>false</el-ignored>
        <scripting-invalid>false</scripting-invalid>
        <include-prelude>tail.jspf</include-prelude>
        <trim-directive-whitespaces>true</trim-directive-whitespaces>
    </jsp-property-group>
</jsp-config>
```

上面的属性组定义了起作用的jsp文件，其包含的片段-tail.jspf，以及去除空白行、一些关于EL和Scriplet的设置。注意，忽略EL表达式后，其会被原样打印。

## 覆盖 jspInit 方法以初始化类

```jsp
<%! //这是一个声明
    //////用来展示JSP初始化覆盖的方法//////
    public void jspInit() {
        //这里还是传统的Servlet地盘，因此可以用这些传统Servlet的方法
        ServletConfig config = getServletConfig();
        String database = config.getInitParameter("Database");
        ServletContext context = getServletConfig().getServletContext();
        synchronized (context) { context.setAttribute("Database",database); }
    }
%>
```

## 设置 page 指令与使用表达式

```jsp
<!-- 这是一个HTML的注释，这句话会被write到out里。-->
<%-- 这是JSP的注释，不会被当作HTML注释生成页面。注意page指令中，我们添加了常见的部分，包导入、EL语言解析、页面编码、错误设置
以及会话管理--%>
<%@ page import="java.util.*"  isELIgnored="false"
         contentType="text/html;charset=UTF-8" language="java"
         session="true" isErrorPage="false" errorPage="" %>
<html>
<head><title>Hello <%= "Corkine"%> <%--这是一个表达式--%></title></head>
<body>
```


## 自定义 jspService 以提供服务

下列代码用来展示JSP的隐藏对象及其使用方法，包括 request、response、config、application、session、out 以及 page 和 pageContext。

```jsp
<%  //这是一个Scriptlet

request.getRequestURI();
response.addHeader("jsp","jsp!!!");
config.getServletName();
//config指的是 getServletConfig() 获得的 ServletConfig 对象
application.getContextPath();
//application 指的是 ServletContext对象，对于Servlet使用 getServletConfig().getServletContext() 获取
session.setAttribute("a","value_of_a");
//session 指的是 request.getSession() 获取的 HttpSession 对象
out.print(application.getAttribute("Database")); //out不是 response.getWriter() 得到的 Writer。而是 jsp.JspWriter
//它的大部分方法和PrintWriter相同，不过添加了一些缓冲功能

//除了这些不同，JSP有一个很强大的pageContext，这是继承自JspContext的类
pageContext.getAttribute("a"); //页面作用域，等同于 request.getAttribute("a")
pageContext.getAttribute("a",PageContext.PAGE_SCOPE); //页面作用域，同上，类似于Servlet作用域
pageContext.getAttribute("a",PageContext.SESSION_SCOPE); //会话作用域，等同于 session.getAttribute("a");
pageContext.getAttribute("a",PageContext.APPLICATION_SCOPE); //APP作用域，等同于 application.getAttribute("a")
pageContext.findAttribute("b"); //从页面/Servlet开始找，到会话，到APP，找到即停止
//set等同，这里不再展示
%>
```

更新记录：

2018年9月16日 开始撰写本文，添加 MVC 和 HTTP 部分

2018年9月19日 Servlet部分 和 Listener部分

2018年9月20日 Session部分 和 JSP部分